In [1]:

import requests
import json
import os
import numpy as np
import base64
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import joblib



# Constants
RSA_SIZE = 2048
DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]
DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"
server = "https://127.0.0.1:8000"
num_users = 4
url = server + "/app/api/metrics"
workspace_path = "workspace"

# Helper function to get file path
def get_workspace_path(file_name):
    return os.path.join(os.getcwd(), workspace_path, file_name)

# Paths to certificate and key files
service_cert_path = get_workspace_path("sandbox_common/service_cert.pem")
user0_cert_path = get_workspace_path("sandbox_common/user0_cert.pem")
user0_privk_path = get_workspace_path("sandbox_common/user0_privk.pem")
user1_cert_path = get_workspace_path("sandbox_common/user1_cert.pem")
user1_privk_path = get_workspace_path("sandbox_common/user1_privk.pem")
member0_cert_path = get_workspace_path("sandbox_common/member0_cert.pem")
member0_privk_path = get_workspace_path("sandbox_common/member0_privk.pem")
    

2024-06-20 19:56:15.328461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 19:56:15.643984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 19:56:15.645870: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 19:56:16.869134: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

# Check server health
def checkServerHealth():
    try:
        response = requests.get(f'{server}/app/status', verify=service_cert_path)
        if response.status_code == 200:
            print("Server is healthy.")
        else:
            print(f"Server is not healthy. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print("Error making request:", e)

checkServerHealth()

# Get response from server
try:
    response = requests.get(url, verify=service_cert_path)

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)
    

Server is healthy.
Status Code: 200

Response Headers:
content-length: 347
content-type: application/json

Response Body:
{
    "metrics": [
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "api/metrics",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "commit",
            "retries": 0
        },
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "POST",
            "path": "model/intial_model",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "status",
            "retries": 0
        }
    ]
}


In [3]:

import os
print(os.getcwd())
!ls -l {os.getcwd()}

/home/aamar/Downloads/project/CCF_FL_Block
total 1800
drwxrwxr-x 3 aamar aamar    4096 Jun 20 19:48 build
drwxrwxrwx 2 aamar aamar    4096 Jun 12 14:15 cmake
-rwxrwxrwx 1 aamar aamar    1752 Jun 12 14:15 CMakeLists.txt
drwxrwxrwx 2 aamar aamar    4096 Jun 12 14:15 config
drwxrwxrwx 3 aamar aamar    4096 Jun 12 14:15 cpp
drwxrwxrwx 2 aamar aamar    4096 Jun 12 14:15 docker
-rwxrwxrwx 1 aamar aamar     179 Jun 12 14:15 Dockerfile
-rwxrwxrwx 1 aamar aamar     150 Jun 12 14:15 Dockerfile.ignore
-rwxrwxrwx 1 aamar aamar     750 Jun 12 14:15 Dockerfile.virtual
drwxrwxrwx 2 aamar aamar    4096 Jun 20 19:32 experiments
-rw-rw-r-- 1 aamar aamar   19649 Jun 20 19:45 federated_learning_notebook.ipynb
-rwxrwxrwx 1 aamar aamar 1387083 Jun 12 14:15 FL_Clients.ipynb
-rwxrwxrwx 1 aamar aamar    1161 Jun 12 14:15 LICENSE
-rwxrwxrwx 1 aamar aamar    4440 Jun 12 14:15 Makefile
-rwxrwxrwx 1 aamar aamar   53243 Jun 13 00:27 mnistExp.ipynb
-rwxrwxrwx 1 aamar aamar     320 Jun 12 14:15 oe_sign.conf
-rwxrwxrw

In [4]:

# Function to compute gradients
def compute_gradients(model, X, y):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        loss = tf.keras.losses.categorical_crossentropy(y, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    return gradients

# Function to aggregate weights
def aggregate_weight(client_weights_list):
    if client_weights_list:
        total_weights = sum(client_weights_list, [])
        aggregated_weights = [weight / len(client_weights_list) for weight in total_weights]
        return aggregated_weights
    else:
        return []

# Function to create the model
def create_model():
    model = Sequential([
        Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, kernel_size=3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to aggregate weights
def aggregate_weights(model_id, round_no, user_cert, user_key):
    response = requests.put(
        url=f"{server}/app/model/aggregate_weights_local?model_id={model_id}&round_no={round_no}",
        verify=service_cert_path,
        cert=(user_cert, user_key)
    )
    if response.status_code == 200:
        print("Aggregation successful for model:", model_id)
    else:
        raise Exception(f"Failed to aggregate weights. Status code: {response.status_code}")

# Function to train the model
def train_model(model, X_train, y_train, X_test, y_test, user_id, round_no, epochs=1):
    batch_size = 16
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
    return history.history['loss']

# Function to serialize the model
def serialize_model(model):
    with open('temp_model.pkl', 'wb') as file:
        joblib.dump(model, file)
    with open('temp_model.pkl', 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Function to upload the initial model
def upload_initial_model(model_base64, user_cert, user_key):
    payload = {
        "global_model": {
            "model_name": "CNNModel",
            "model_data": model_base64
        }
    }
    response = requests.post(
        url=f"{server}/app/model/intial_model",
        verify=service_cert_path,
        cert=(user_cert, user_key),
        json=payload
    )

    if response.status_code == 200:
        model_data = response.json()
        model_id = model_data.get("model_id")
        model_name = model_data.get("model_name")
        print(f"Initial global model '{model_name}' (ID: {model_id}) uploaded successfully.")
        return model_id
    else:
        print(f"Failed to upload initial model. Status code: {response.status_code}")
        return None

# Function to flatten weights
def flatten_weights(model):
    flat_weights = []
    for layer in model.layers:
        weights = layer.get_weights()
        if weights:
            flat_weights.append(weights[0].flatten())
    return np.concatenate(flat_weights)

# Function to deserialize weights
def deserialize_weights(serialized_weights, model):
    flat_weights = np.array(serialized_weights)
    unflattened_weights = unflatten_weights(model, flat_weights)
    return unflattened_weights

# Function to unflatten weights
def unflatten_weights(model, flat_weights):
    unflattened_weights = []
    index = 0
    for layer in model.layers:
        layer_weights = layer.get_weights()
        if layer_weights:
            weights_shape = layer_weights[0].shape
            layer_weights_unflattened = flat_weights[index:index + np.prod(weights_shape)].reshape(weights_shape)
            unflattened_weights.append(layer_weights_unflattened)
            index += np.prod(weights_shape)
    unflattened_weights = [np.array(arr) for arr in unflattened_weights]
    return unflattened_weights

# Function to serialize gradients
def serialize_gradients(gradients):
    serialized_gradients = [grad.tolist() for grad in gradients]
    return json.dumps(serialized_gradients)

# Function to deserialize gradients
def deserialize_gradients(serialized_gradients):
    gradients_list = json.loads(serialized_gradients)
    return [np.array(grad) for grad in gradients_list]

# Function to upload gradients
def upload_gradients(gradients_base64, user_cert, user_key, round_no, model_id=None):
    print(f"Uploading gradients for Round {round_no}...")
    payload = {
        "model_id": model_id,
        "gradients_json": gradients_base64,
        "round_no": round_no
    }
    response = requests.post(
        url=f"{server}/app/model/upload/local_gradients",
        verify=service_cert_path,
        cert=(user_cert, user_key),
        json=payload
    )
    if response.status_code == 200:
        print(f"Gradients uploaded successfully for Round {round_no}.")
    else:
        raise Exception(f"Failed to upload gradients. Status code: {response.status_code}")

# Function to download global gradients
def download_global_gradients(user_cert, user_key, model_id):
    try:
        response = requests.get(
            url=f"{server}/app/model/download_global_gradients?model_id={model_id}",
            verify=service_cert_path,
            cert=(user_cert, user_key)
        )
        if response.status_code == 200:
            print("Global gradients downloaded successfully.")
            response_data = response.json()
            global_gradients_value = response_data.get("global_gradients")
            if global_gradients_value:
                gradients = deserialize_gradients(global_gradients_value)
                return gradients
            else:
                print("Global gradients data not found in response.")
        else:
            print(f"Failed to download global gradients. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print("Error making request:", e)
    return None

# Function to apply gradients
def apply_gradients(model, gradients):
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Function to delete temporary model file
def delete_temp_model_file():
    if os.path.exists('temp_model.pkl'):
        os.remove('temp_model.pkl')

# Function to plot loss curve
def plot_loss_curve(round_loss_dict):
    rounds = list(round_loss_dict.keys())
    losses_user0 = [round_loss_dict[round][0] for round in rounds]
    losses_user1 = [round_loss_dict[round][1] for round in rounds]

    plt.plot(rounds, losses_user0, label='User 0 Loss')
    plt.plot(rounds, losses_user1, label='User 1 Loss')

    plt.xlabel('Round Number')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curve for Each User')
    plt.grid()
    plt.show()
    

: 

In [5]:

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

num_users = 2
user_split_size = X_train.shape[0] // num_users
X_train_user0 = X_train[:user_split_size]
y_train_user0 = y_train[:user_split_size]
X_train_user1 = X_train[user_split_size:user_split_size*2]
y_train_user1 = y_train[user_split_size:user_split_size*2]

global_model = create_model()

serialized_model = serialize_model(global_model)
initial_model_id = upload_initial_model(serialized_model, user0_cert_path, user0_privk_path)

num_rounds = 5
round_loss_dict = {}

local_model_user0 = global_model
local_model_user1 = global_model

for round_no in range(1, num_rounds + 1):
    round_loss_dict[round_no] = {}
    for user_id in range(2):
        X_train_user = X_train_user0 if user_id == 0 else X_train_user1
        y_train_user = y_train_user0 if user_id == 0 else y_train_user1
        gradients = compute_gradients(local_model_user0 if user_id == 0 else local_model_user1, X_train_user, y_train_user)
        serialized_gradients = serialize_gradients(gradients)
        print(f"User {user_id} - Round {round_no} - Gradients Length: {len(serialized_gradients)}")
        upload_gradients(serialized_gradients, user0_cert_path if user_id == 0 else user1_cert_path, user0_privk_path if user_id == 0 else user1_privk_path, round_no, model_id=initial_model_id)
    
    global_gradients = download_global_gradients(user0_cert_path, user0_privk_path, model_id=initial_model_id)
    if global_gradients:
        apply_gradients(local_model_user0, global_gradients)
        apply_gradients(local_model_user1, global_gradients)
    else:
        print("No global gradients received for this round.")

print("Federated Learning Process Completed.")
delete_temp_model_file()
plot_loss_curve(round_loss_dict)
    

Initial global model 'CNNModel' (ID: 1) uploaded successfully.


2024-06-20 19:56:19.768242: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5191680000 exceeds 10% of free system memory.
2024-06-20 19:56:21.248675: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5191680000 exceeds 10% of free system memory.
